In [ ]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
import os
from tqdm.notebook import tqdm

In [ ]:
# Download the TIMIT dataset
!wget cls.ru.nl/~ltenbosch/TIMIT.tar.gz
!tar -zxvf /content/TIMIT.tar.gz

In [ ]:
# Initialize empty lists for train and test sets
mfcc_train = []
mfcc_test = []

# Define the root directories for train and test sets
root_dir_train = 'wavfiles16kHz/TRAIN'
root_dir_test = 'wavfiles16kHz/TEST'

# Iterate over the train files
for filename in tqdm(os.listdir(root_dir_train)):
    if filename.endswith('.WAV'):
        file_path = os.path.join(root_dir_train, filename)
        audio, sr = librosa.load(file_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr)
        mfcc_train.append(np.mean(mfcc, axis=1))

# Iterate over the test files
for filename in tqdm(os.listdir(root_dir_test)):
    if filename.endswith('.WAV'):
        file_path = os.path.join(root_dir_test, filename)
        audio, sr = librosa.load(file_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr)
        mfcc_test.append(np.mean(mfcc, axis=1))

  0%|          | 0/4620 [00:00<?, ?it/s]

  0%|          | 0/1680 [00:00<?, ?it/s]

In [ ]:
labels_train = []
root_dir = 'wavfiles16kHz/TRAIN'
for filename in os.listdir(root_dir):
    if filename.endswith('.WAV'):
        label = filename[3]
        labels_train.append(label)

labels_test = []
root_dir = 'wavfiles16kHz/TEST'
for filename in os.listdir(root_dir):
    if filename.endswith('.WAV'):
        label = filename[3]
        labels_test.append(label)


# Convert the labels to one-hot encoding
encoder = LabelEncoder()
# Fit the encoder on the train labels
encoder.fit(labels_train)
# Transform the train and test labels to one-hot encoding
encoded_labels_train = encoder.transform(labels_train)
encoded_labels_test = encoder.transform(labels_test)
one_hot_labels_train = to_categorical(encoded_labels_train)
one_hot_labels_test = to_categorical(encoded_labels_test)

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=mfcc_train[0].shape[0], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(one_hot_labels_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Convert the lists to NumPy arrays and reshape
mfcc_train = np.array(mfcc_train)
mfcc_test = np.array(mfcc_test)

In [ ]:
# Train the model
model.fit(mfcc_train, one_hot_labels_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
58/58 [==============================] - 2s 7ms/step - loss: 2.7459 - accuracy: 0.6718 - val_loss: 0.5364 - val_accuracy: 0.8160
Epoch 2/10
58/58 [==============================] - 0s 4ms/step - loss: 0.2871 - accuracy: 0.8831 - val_loss: 0.2036 - val_accuracy: 0.9199
Epoch 3/10
58/58 [==============================] - 0s 4ms/step - loss: 0.1563 - accuracy: 0.9364 - val_loss: 0.1439 - val_accuracy: 0.9491
Epoch 4/10
58/58 [==============================] - 0s 4ms/step - loss: 0.1293 - accuracy: 0.9478 - val_loss: 0.1417 - val_accuracy: 0.9437
Epoch 5/10
58/58 [==============================] - 0s 5ms/step - loss: 0.1151 - accuracy: 0.9537 - val_loss: 0.1099 - val_accuracy: 0.9643
Epoch 6/10
58/58 [==============================] - 0s 4ms/step - loss: 0.1072 - accuracy: 0.9578 - val_loss: 0.1556 - val_accuracy: 0.9459
Epoch 7/10
58/58 [==============================] - 0s 4ms/step - loss: 0.1095 - accuracy: 0.9578 - val_loss: 0.0957 - val_accuracy: 0.9621
Epoch 8/10
58/58 [==

In [ ]:
# Test the model
score = model.evaluate(mfcc_test, one_hot_labels_test) # Use one_hot_labels_test instead of y_test
print("Accuracy:", score[1])

53/53 [==============================] - 0s 3ms/step - loss: 0.2404 - accuracy: 0.9179
Accuracy: 0.9178571701049805
